## Assignment 3:  Convolutional networks for TF ChIP-seq data

In this assignment you will build on the convolutional networks we looked at in class and work on ChIP-seq data for four transcription factors in arabidopsis.


### Part 1: Data Preparation

In this assignment you will work with ChIP-seq for four arabidopsis transcription factors:  AGL16, GRF1, AMS, and MYB3R4.  The peaks that represent their binding sites in the arabidopsis genome are available in the following links:

* AGL16 ([bed file](https://biobigdata.nju.edu.cn/ChIPHub_download/arabidopsis_thaliana/SRP187795/hammock/AGL16.target.all.bed.gz))
* GRF1 ([bed file](https://biobigdata.nju.edu.cn/ChIPHub_download/arabidopsis_thaliana/SRP002566/hammock/SRX021610.peak.all.bed.gz))
* AMS ([bed file](https://biobigdata.nju.edu.cn/ChIPHub_download/arabidopsis_thaliana/SRP188198/hammock/SRX5507861.peak.all.bed.gz))
* MYB3R4 ([bed file](https://biobigdata.nju.edu.cn/ChIPHub_download/arabidopsis_thaliana/SRP244735/hammock/SRX7616358.peak.all.bed.gz)).

These files are in [bed format](https://en.wikipedia.org/wiki/BED_(file_format)), and contain the information on the genomic locations where the ChIP-seq peaks have been detected.  The linked wikipedia article provides the information you need about the format of these files.  Your task is to extract sequences of length 500 centered at the location of each peak, which you will provide as input to the convolutional network you train.  

The other piece of information you need is the genomic sequence for arabidopsis.  This is available from the [Ensembl plants arabidopsis portal](https://plants.ensembl.org/Arabidopsis_thaliana/Info/Index).  In that page click on "Download DNA sequence (FASTA)", and the first five files provide the sequences for the five arabidopsis chromosomes.
For reference, we computed the result for AGL16 (link is in the assignment page in Canvas).

Your final data preparation task is to prepare a labeled dataset with positive examples that correspond to the peak sequences.  As negative examples, use random permutations of the positive examples.  Create one permutation from each positive example.

In [1]:
from numpy import random
import os

import pandas as pd

import numpy as np

path='/mnt/c/Users/hunte/Desktop/CS525'
#path=r'C:\Users\hunte\Desktop\CS525'
os.chdir(path)

import warnings
warnings.filterwarnings(action='once')


import pybedtools #import Bedtool

usingLinux=False

#path=r'C:\\Users\\hunte\Desktop\\CS525'
#path='/mnt/c/Users/hunte/Desktop/CS525'
#os.chdir(path)
#Loop through BED file

#bedFiles = ['AGL16.target.all.bed','SRX021610.peak.all.bed','SRX5507861.peak.all.bed','SRX7616358.peak.all.bed']

bedFiles = ['RNAPosProm.txt','RNANegProm.txt']

#Read bed line by line, taking first three numbers

posSeqs = []
#This will be a x by y array, where x is the number of bed files with peak data and y is the number of sequences in a single bed file
#Each element will be a string of 100 characters representing a positive peak for a given sample

#def getSeq(chrom,start,end):

#    return

#ColumnNames = ['AGL16_Pos','AGL16_Neg','GRF1_Pos','GRF1_Neg','AMS_Pos','AMS_Neg','MYR3R4_Pos','MYR3R4_Neg']

#ColumnNames = ['Ac_Pos','Ac_Neg','Me_Pos','Me_Neg']

ColumnNames = ['DiffSeqs','ControlSeqs']

tfNames = ['H3K9Me3','H3K27Ac']

fileNum=0
for file in bedFiles:
    filePos = []
    bedFile = open(file,'r')
    bedLines = bedFile.readlines()
    #temp = 0
    fastaFile = open(tfNames[fileNum]+'_HO_peaks.fasta','w')
    for line in bedLines:
        fields = line.split()
        midPeak = round((int(fields[1])+int(fields[2]))/2)

        #SWITCH BACK FOR CS525 ASSIGNMENT

        peakLow = midPeak-750
        peakHigh = midPeak+750
        #if(peakLow < 0):
            #peakLow = 0
            #peakHigh = 500
        #position = str(fields[0][3]) + "\t" + str(fields[1]) + "\t" + str(fields[2])
        #position = str(fields[0][3]) + "\t" + str(peakLow) + "\t" + str(peakHigh)

        ####### CHANGE TO PREVIOUS FOR CS525

        position = str(fields[0]) + "\t" + str(peakLow) + "\t" + str(peakHigh)

        #Need to account for possibility of extending beyond edge of chromosomes

        #print(position)
        posBed = pybedtools.BedTool(position,from_string=True)
        try:
            posSeq = posBed.sequence(fi="AedesGenome68.fa")
        except:
            continue
        #print(posSeq)
        test = open(posSeq.seqfn)
        print(test)
        #print(open(posSeq.seqfn).read().split()[1])
        a = test.read().split()[1]
        #a = test.read()#.split()[1]
        #print(a)
        #posSeq.seqfn.close()
        labelString = ">" + tfNames[fileNum] + ',' + fields[0] + "," + str(peakLow) + ',' + str(peakHigh) + ',500'
        filePos.append(a)
        test.close()
        fastaFile.write(labelString+'\n')
        fastaFile.write(a+'\n')
        #temp=temp+1
        #if(temp > 10):
        #    break
    fastaFile.close()
    bedFile.close()
    posSeqs.append(filePos)
    fileNum=fileNum+1


#Get positive example from appopriate chromosome as a string

#pos_example =  

#Generate permutation of sequence to use as a negative example

negSeqs = []

for pos in posSeqs:
    fileNeg = []
    for seq in pos:
        #print(seq)
        #neg_example = random.permutation(seq)
        #print(seq)
        #print(random.shuffle(list(seq)))
        #neg_example = str(random.shuffle(list(seq)))
        negSeq = list(seq)
        #print(seq)
        #print(negSeq)
        random.seed(2024)
        random.shuffle(negSeq)
        #print(negSeq)
        negSeq = "".join(negSeq)
        #print(negSeq)
        #neg_example = seq
        #Can simplify after debugging
        fileNeg.append(negSeq)
    negSeqs.append(fileNeg)

#Add both to data frame

#ColumnNames = ['Ac_Pos','Ac_Neg','Me_Pos','Me_Neg']
ColumnNames = ['DiffRNA','ControlRNA']

#samplesFrame = pd.DataFrame([posSeqs[0],negSeqs[0],posSeqs[1],negSeqs[1],
#                             posSeqs[2,negSeqs[2],posSeqs[3],negSeqs[3]]],columns=ColumnNames)

#for i in seq(4):


#negSeqs.shape()

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/mnt/c/Users/hunte/Desktop/CS525'

In [ ]:
import pickle
posSave = open("PosExamplesRNAProm.pkl", 'wb') 
pickle.dump(posSeqs, posSave)
negSave = open("NegExamplesRNAProm.pkl", 'wb') 
pickle.dump(negSeqs, negSave)

negSave.close()
posSave.close()



In [14]:
'''import pickle

testFile="PosExamples.pkl"
#posSave = open("Assignment3DataRead.pkl", 'wb')
posSeqs=pd.read_pickle(testFile)

testFile="NegExamples.pkl"
negSeqs=pd.read_pickle(testFile)


#testFile="PosExamples.pkl"
#pickle.dump(dataset, dataSave)
#dataSave.close()'''

In [17]:
'''import itertools

DFNames = ['Transcription Factor','Event_ID','Seq','Bound']

#tfNames = ['AGL16','GRF1','AMS','MYR3R4']

tfNames = ['H3K9Me3','H3K27Ac']


for i in range(4):
    #trainFile = open(tfNames[i]+'_HO.seq','w')
    
    seqNum = len(posSeqs[i])
    events = list(range(2*(seqNum-1)))
    eventNums = ["seq_"+str(event)+"_peak" for event in events]
    TF_name = list(itertools.repeat(tfNames[i],2*(seqNum-1)))
    bound = list(itertools.repeat(1,seqNum-1))+list(itertools.repeat(0,seqNum-1))
    allSeqs = posSeqs[i][:] + negSeqs[i][:]
    #allSeqs.append(negSeqs[i])
    #allSeqs = posSeqs[i].append(negSeqs[i])
    #print(len(TF_name))
    #print(len(allSeqs))
    #print(len(eventNums))
    #print(len(bound))


    tempFrame = pd.DataFrame(zip(*(TF_name,eventNums,allSeqs,bound)),columns=DFNames)
    #tempFrame.reset_index(drop=True,inplace=True)
    tempFrame.to_csv(tfNames[i]+'_HO.seq',sep='\t',index=False)
    #trainFile.write(labelString+'\n')
    #trainFile.write(a+'\n')
    #trainFile.close()'''



### Part 2:  

As discussed in class, deeper networks with multiple layers of convolution *can* improve a network's performance.  Your task here is to extend the implementation provided in class to have three layers of convolution.  In addition, implement early stopping based on performance on the validation set.
Finally, train each network two or three times, and choose the best performing network (based on the performance on the validation set) as the one to evaluate on the test set.
In your experiments, set aside 20% of the data for testing, 20% for validation, and 60% for training.
Compare the accuracy of your network to that of a one layer CNN.  Accuracy should be measured using the area under the ROC curve.  In the next part of the assignment you will get to tune its parameters to try and improve its performance.

In this part, you can focus on two out of the four datasets provided above.
